# 💰 TP3 : Modélisation Exposition Économique Abidjan

## Formation DGE Côte d'Ivoire - Jour 3

### Objectifs pédagogiques :
- Créer exposition économique détaillée et réaliste
- Intégrer données sectorielles (INS, BCEAO)
- Géocoder et spatialiser les actifs économiques
- Calibrer valeurs avec statistiques nationales
- Produire cartes d'exposition haute résolution

### Durée : 2h30
### Niveau : Intermédiaire

---

## 📚 Étape 1 : Import et Données de Base

In [ ]:
# Imports essentiels
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Imports CLIMADA
from climada.entity import Exposures
from climada.hazard import Hazard
from climada.util.coordinates import coord_on_land

# Imports géospatial
from shapely.geometry import Point, Polygon
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

# Configuration
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

print("✅ Imports réussis - Prêt pour modélisation exposition!")
print(f"📍 Répertoire de travail : {Path.cwd()}")

In [ ]:
# Données économiques officielles Côte d'Ivoire (2023)
# Sources : INS, BCEAO, Banque Mondiale (données réalistes)

donnees_macro_ci = {
    'pib_national': 70000e9,  # 70 000 Mds FCFA
    'pib_abidjan_pct': 0.42,  # 42% du PIB national
    'population_nationale': 28_000_000,
    'population_abidjan': 6_500_000,
    'taux_croissance': 0.065,  # 6.5% croissance 2023
    'inflation': 0.04,  # 4% inflation
    'taux_change_usd': 580,  # 1 USD = 580 FCFA
}

# Calculs dérivés
pib_abidjan = donnees_macro_ci['pib_national'] * donnees_macro_ci['pib_abidjan_pct']
pib_per_capita_abidjan = pib_abidjan / donnees_macro_ci['population_abidjan']

print("🏛️ DONNÉES MACROÉCONOMIQUES CÔTE D'IVOIRE 2023")
print("=" * 55)
print(f"PIB National : {donnees_macro_ci['pib_national']/1e12:.1f} billions FCFA")
print(f"PIB Abidjan : {pib_abidjan/1e12:.1f} billions FCFA ({donnees_macro_ci['pib_abidjan_pct']*100:.0f}%)")
print(f"Population Abidjan : {donnees_macro_ci['population_abidjan']:,} habitants")
print(f"PIB/habitant Abidjan : {pib_per_capita_abidjan/1e6:.2f} millions FCFA/an")
print(f"PIB/habitant Abidjan : {pib_per_capita_abidjan/donnees_macro_ci['taux_change_usd']:,.0f} USD/an")

In [ ]:
# Données sectorielles détaillées Abidjan (basées sur études INS)
secteurs_economiques = {
    'Agriculture_Peche': {
        'part_pib': 0.08,  # 8% du PIB d'Abidjan
        'emplois': 520000,
        'productivite': 'faible',
        'croissance': 0.03,
        'zones': ['Port-Bouet', 'Yopougon_Sud', 'Bingerville']
    },
    'Industries_Manufacturieres': {
        'part_pib': 0.28,  # 28% - secteur dominant
        'emplois': 850000,
        'productivite': 'elevee',
        'croissance': 0.08,
        'zones': ['Yopougon', 'Koumassi', 'Marcory', 'Abobo']
    },
    'BTP': {
        'part_pib': 0.12,
        'emplois': 780000,
        'productivite': 'moyenne',
        'croissance': 0.10,  # Boom construction
        'zones': ['Plateau', 'Cocody', 'Marcory', 'Treichville']
    },
    'Commerce_Services': {
        'part_pib': 0.35,  # 35% - services tertiaires
        'emplois': 2100000,
        'productivite': 'elevee',
        'croissance': 0.07,
        'zones': ['Plateau', 'Adjame', 'Treichville', 'Cocody']
    },
    'Transport_Logistique': {
        'part_pib': 0.10,  # Port autonome + aéroport
        'emplois': 420000,
        'productivite': 'elevee',
        'croissance': 0.06,
        'zones': ['Port-Bouet', 'Treichville', 'Marcory']
    },
    'Services_Publics': {
        'part_pib': 0.07,
        'emplois': 320000,
        'productivite': 'moyenne',
        'croissance': 0.04,
        'zones': ['Plateau', 'Cocody', 'Deux-Plateaux']
    }
}

print("\n📊 STRUCTURE ÉCONOMIQUE SECTORIELLE ABIDJAN")
print("=" * 55)

total_emplois = sum(s['emplois'] for s in secteurs_economiques.values())
total_part_pib = sum(s['part_pib'] for s in secteurs_economiques.values())

for secteur, data in secteurs_economiques.items():
    valeur_ajoutee = pib_abidjan * data['part_pib']
    productivite_emploi = valeur_ajoutee / data['emplois']
    
    print(f"{secteur:25s}: {data['part_pib']*100:4.1f}% PIB, "
          f"{data['emplois']:,} emplois, "
          f"{productivite_emploi/1e6:.1f}M FCFA/emploi")

print(f"\nTotal emplois estimés : {total_emplois:,}")
print(f"Vérification part PIB : {total_part_pib:.2f} (doit être ≈ 1.0)")

## 🗺️ Étape 2 : Géographie Administrative et Économique

In [ ]:
# Définir géographie détaillée d'Abidjan par commune
# Coordonnées réelles et caractéristiques socio-économiques

communes_abidjan = {
    'Plateau': {
        'centre': [5.325, -4.024],  # Centre administratif
        'superficie': 2.5,  # km²
        'population': 15000,
        'densite': 6000,  # hab/km²
        'revenus_moyens': 8500000,  # FCFA/an/hab
        'secteur_dominant': 'Services_Publics',
        'caracteristiques': ['CBD', 'banques', 'administrations'],
        'vulnerabilite': 'faible'  # Infrastructure moderne
    },
    'Cocody': {
        'centre': [5.347, -3.987],
        'superficie': 54.0,
        'population': 687000,
        'densite': 12722,
        'revenus_moyens': 2800000,
        'secteur_dominant': 'Commerce_Services',
        'caracteristiques': ['residentiel_haut', 'universites', 'ambassades'],
        'vulnerabilite': 'faible'
    },
    'Yopougon': {
        'centre': [5.345, -4.108],
        'superficie': 153.0,
        'population': 1200000,
        'densite': 7843,
        'revenus_moyens': 1200000,
        'secteur_dominant': 'Industries_Manufacturieres',
        'caracteristiques': ['industriel', 'habitat_populaire', 'marches'],
        'vulnerabilite': 'elevee'
    },
    'Abobo': {
        'centre': [5.416, -4.020],
        'superficie': 69.0,
        'population': 1300000,
        'densite': 18841,
        'revenus_moyens': 1000000,
        'secteur_dominant': 'Industries_Manufacturieres',
        'caracteristiques': ['habitat_precaire', 'forte_densite', 'jeune'],
        'vulnerabilite': 'tres_elevee'
    },
    'Adjame': {
        'centre': [5.366, -4.052],
        'superficie': 14.0,
        'population': 450000,
        'densite': 32143,
        'revenus_moyens': 2200000,
        'secteur_dominant': 'Commerce_Services',
        'caracteristiques': ['commerce', 'transport', 'gare_routiere'],
        'vulnerabilite': 'elevee'
    },
    'Marcory': {
        'centre': [5.293, -4.003],
        'superficie': 44.0,
        'population': 500000,
        'densite': 11364,
        'revenus_moyens': 1800000,
        'secteur_dominant': 'Industries_Manufacturieres',
        'caracteristiques': ['industriel', 'port', 'residentiel_moyen'],
        'vulnerabilite': 'moyenne'
    },
    'Treichville': {
        'centre': [5.293, -4.024],
        'superficie': 15.0,
        'population': 280000,
        'densite': 18667,
        'revenus_moyens': 2000000,
        'secteur_dominant': 'Commerce_Services',
        'caracteristiques': ['commerce', 'port', 'finance'],
        'vulnerabilite': 'moyenne'
    },
    'Koumassi': {
        'centre': [5.294, -3.954],
        'superficie': 32.0,
        'population': 430000,
        'densite': 13438,
        'revenus_moyens': 1400000,
        'secteur_dominant': 'Industries_Manufacturieres',
        'caracteristiques': ['industriel', 'residentiel_populaire'],
        'vulnerabilite': 'elevee'
    },
    'Port-Bouet': {
        'centre': [5.235, -3.931],
        'superficie': 252.0,
        'population': 320000,
        'densite': 1270,
        'revenus_moyens': 1600000,
        'secteur_dominant': 'Transport_Logistique',
        'caracteristiques': ['aeroport', 'port', 'peche', 'tourisme'],
        'vulnerabilite': 'elevee'  # Zone côtière
    },
    'Attecoube': {
        'centre': [5.332, -4.080],
        'superficie': 16.0,
        'population': 380000,
        'densite': 23750,
        'revenus_moyens': 1100000,
        'secteur_dominant': 'Commerce_Services',
        'caracteristiques': ['habitat_populaire', 'artisanat', 'marches'],
        'vulnerabilite': 'tres_elevee'
    }
}

print("🗺️ COMMUNES D'ABIDJAN - PROFILS SOCIO-ÉCONOMIQUES")
print("=" * 65)

population_totale = 0
superficie_totale = 0
pib_commune_total = 0

for commune, data in communes_abidjan.items():
    pop = data['population']
    sup = data['superficie']
    rev = data['revenus_moyens']
    
    # PIB approximatif de la commune
    pib_commune = pop * rev
    
    population_totale += pop
    superficie_totale += sup
    pib_commune_total += pib_commune
    
    print(f"{commune:12s}: {pop:7,} hab, {sup:6.1f} km², "
          f"{data['densite']:5,} hab/km², {rev/1e6:.1f}M FCFA/hab, "
          f"PIB: {pib_commune/1e12:.3f}T FCFA")

print(f"\nTOTAL DISTRICT: {population_totale:,} hab, {superficie_totale:.1f} km²")
print(f"Densité moyenne: {population_totale/superficie_totale:,.0f} hab/km²")
print(f"PIB total estimé: {pib_commune_total/1e12:.2f} billions FCFA")
print(f"Ratio PIB macro/micro: {pib_abidjan/pib_commune_total:.2f}")

In [ ]:
# Créer carte des communes d'Abidjan
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Extraire données pour visualisation
lons = [data['centre'][1] for data in communes_abidjan.values()]
lats = [data['centre'][0] for data in communes_abidjan.values()]
populations = [data['population'] for data in communes_abidjan.values()]
revenus = [data['revenus_moyens'] for data in communes_abidjan.values()]
communes_noms = list(communes_abidjan.keys())

# Carte 1: Population par commune
scatter1 = ax1.scatter(lons, lats, s=[p/5000 for p in populations], 
                      c=populations, cmap='Reds', alpha=0.7, 
                      edgecolors='black', linewidth=1)

for i, commune in enumerate(communes_noms):
    ax1.annotate(commune, (lons[i], lats[i]), 
                xytext=(5, 5), textcoords='offset points', 
                fontsize=9, fontweight='bold')

ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_title('Population par Commune\n(Taille = Population)')
ax1.grid(True, alpha=0.3)
cbar1 = plt.colorbar(scatter1, ax=ax1)
cbar1.set_label('Population')

# Carte 2: Revenus moyens par commune
scatter2 = ax2.scatter(lons, lats, s=[r/50000 for r in revenus], 
                      c=revenus, cmap='Greens', alpha=0.7, 
                      edgecolors='black', linewidth=1)

for i, commune in enumerate(communes_noms):
    ax2.annotate(commune, (lons[i], lats[i]), 
                xytext=(5, 5), textcoords='offset points', 
                fontsize=9, fontweight='bold')

ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude')
ax2.set_title('Revenus Moyens par Commune\n(Taille = Revenus)')
ax2.grid(True, alpha=0.3)
cbar2 = plt.colorbar(scatter2, ax=ax2)
cbar2.set_label('Revenus (FCFA/an)')

plt.tight_layout()
plt.show()

## 💼 Étape 3 : Génération Actifs Économiques Détaillés

In [ ]:
# Générer actifs économiques géolocalisés pour chaque commune et secteur
# Approche bottom-up basée sur données réelles

np.random.seed(123)  # Reproductibilité

# Paramètres de génération
total_points_exposition = 5000  # Points d'exposition à créer
min_valeur_actif = 1e6  # 1 million FCFA minimum
max_valeur_actif = 50e9  # 50 milliards FCFA maximum

print("🏭 GÉNÉRATION ACTIFS ÉCONOMIQUES GÉOLOCALISÉS")
print("=" * 55)

# Structure pour stocker tous les actifs
actifs_economiques = []
compteurs_secteur = {secteur: 0 for secteur in secteurs_economiques.keys()}

for commune_nom, commune_data in communes_abidjan.items():
    print(f"\n🏘️ Commune: {commune_nom}")
    
    # Nombre d'actifs par commune (proportionnel PIB communal)
    pib_commune = commune_data['population'] * commune_data['revenus_moyens']
    part_commune = pib_commune / pib_commune_total
    n_actifs_commune = int(total_points_exposition * part_commune)
    
    print(f"   Actifs à générer: {n_actifs_commune}")
    
    # Centre et rayon de dispersion
    centre_lat, centre_lon = commune_data['centre']
    rayon_km = np.sqrt(commune_data['superficie'] / np.pi)  # Rayon équivalent cercle
    rayon_deg = rayon_km / 111  # Conversion approximative km -> degrés
    
    for i in range(n_actifs_commune):
        # Position aléatoire dans la commune (distribution normale autour du centre)
        lat = np.random.normal(centre_lat, rayon_deg * 0.6)
        lon = np.random.normal(centre_lon, rayon_deg * 0.6)
        
        # Contraindre dans limites raisonnables d'Abidjan
        lat = np.clip(lat, 5.1, 5.6)
        lon = np.clip(lon, -4.5, -3.5)
        
        # Déterminer secteur (avec biais selon secteur dominant commune)
        secteur_dominant = commune_data['secteur_dominant']
        
        # Probabilités sectorielles ajustées
        probs_secteur = {}
        for secteur in secteurs_economiques.keys():
            if secteur == secteur_dominant:
                probs_secteur[secteur] = 0.4  # 40% secteur dominant
            else:
                probs_secteur[secteur] = 0.6 / (len(secteurs_economiques) - 1)
        
        secteur_actif = np.random.choice(list(secteurs_economiques.keys()), 
                                        p=list(probs_secteur.values()))
        
        # Valeur de l'actif selon secteur et commune
        secteur_info = secteurs_economiques[secteur_actif]
        
        # Valeur de base selon productivité secteur
        if secteur_info['productivite'] == 'elevee':
            base_value = np.random.lognormal(np.log(50e6), 1.2)  # 50M FCFA base
        elif secteur_info['productivite'] == 'moyenne':
            base_value = np.random.lognormal(np.log(20e6), 1.0)  # 20M FCFA base
        else:  # faible
            base_value = np.random.lognormal(np.log(8e6), 0.8)   # 8M FCFA base
        
        # Ajustement selon richesse commune
        facteur_commune = commune_data['revenus_moyens'] / 2000000  # Normalisation
        valeur_finale = base_value * facteur_commune
        
        # Contraindre dans limites
        valeur_finale = np.clip(valeur_finale, min_valeur_actif, max_valeur_actif)
        
        # Type d'actif plus spécifique
        types_actifs = {
            'Agriculture_Peche': ['Ferme_aquaculture', 'Cooperative_peche', 'Marche_poisson'],
            'Industries_Manufacturieres': ['Usine_textile', 'Usine_agroalimentaire', 'Atelier_mecanique', 'Entrepot'],
            'BTP': ['Entreprise_construction', 'Carriere', 'Centrale_beton'],
            'Commerce_Services': ['Centre_commercial', 'Banque', 'Hotel', 'Restaurant', 'Bureau'],
            'Transport_Logistique': ['Terminal_transport', 'Entrepot_logistique', 'Station_service'],
            'Services_Publics': ['Hopital', 'Ecole', 'Administration']
        }
        
        type_actif = np.random.choice(types_actifs[secteur_actif])
        
        # Créer enregistrement actif
        actif = {
            'id': len(actifs_economiques) + 1,
            'latitude': lat,
            'longitude': lon,
            'valeur': valeur_finale,
            'secteur': secteur_actif,
            'type_actif': type_actif,
            'commune': commune_nom,
            'vulnerabilite': commune_data['vulnerabilite'],
            'densite_pop': commune_data['densite'],
            'revenus_zone': commune_data['revenus_moyens']
        }
        
        actifs_economiques.append(actif)
        compteurs_secteur[secteur_actif] += 1

print(f"\n✅ Génération terminée: {len(actifs_economiques)} actifs créés")
print(f"\n📊 Répartition par secteur:")
for secteur, count in compteurs_secteur.items():
    valeur_totale = sum(a['valeur'] for a in actifs_economiques if a['secteur'] == secteur)
    print(f"   {secteur:25s}: {count:4d} actifs, {valeur_totale/1e12:.3f}T FCFA")

valeur_totale_exposition = sum(a['valeur'] for a in actifs_economiques)
print(f"\n💰 Valeur totale exposition: {valeur_totale_exposition/1e12:.2f} billions FCFA")
print(f"📊 Ratio avec PIB Abidjan: {valeur_totale_exposition/pib_abidjan:.2f}")

In [ ]:
# Convertir en DataFrame et GeoDataFrame pour analyse
df_actifs = pd.DataFrame(actifs_economiques)

print("📋 STATISTIQUES DESCRIPTIVES EXPOSITION")
print("=" * 45)

print(f"Nombre total d'actifs: {len(df_actifs):,}")
print(f"Valeur moyenne: {df_actifs['valeur'].mean()/1e6:.1f} millions FCFA")
print(f"Valeur médiane: {df_actifs['valeur'].median()/1e6:.1f} millions FCFA")
print(f"Écart-type: {df_actifs['valeur'].std()/1e6:.1f} millions FCFA")
print(f"Min/Max: {df_actifs['valeur'].min()/1e6:.1f} / {df_actifs['valeur'].max()/1e6:.1f} millions FCFA")

# Statistiques par commune
print(f"\n📊 Répartition par commune:")
stats_commune = df_actifs.groupby('commune').agg({
    'valeur': ['count', 'sum', 'mean'],
    'latitude': 'mean',
    'longitude': 'mean'
}).round(2)

stats_commune.columns = ['Nb_actifs', 'Valeur_totale', 'Valeur_moyenne', 'Lat_centre', 'Lon_centre']

for commune in stats_commune.index:
    row = stats_commune.loc[commune]
    print(f"   {commune:12s}: {row['Nb_actifs']:4.0f} actifs, "
          f"{row['Valeur_totale']/1e12:.3f}T FCFA, "
          f"moy: {row['Valeur_moyenne']/1e6:.1f}M FCFA")

# Statistiques par secteur
print(f"\n📊 Répartition par secteur:")
stats_secteur = df_actifs.groupby('secteur').agg({
    'valeur': ['count', 'sum', 'mean']
}).round(2)

stats_secteur.columns = ['Nb_actifs', 'Valeur_totale', 'Valeur_moyenne']

for secteur in stats_secteur.index:
    row = stats_secteur.loc[secteur]
    print(f"   {secteur:25s}: {row['Nb_actifs']:4.0f} actifs, "
          f"{row['Valeur_totale']/1e12:.3f}T FCFA")

## 🎯 Étape 4 : Création Objet Exposures CLIMADA

In [ ]:
# Créer objet Exposures CLIMADA compatible
print("🏗️ CRÉATION OBJET EXPOSURES CLIMADA")
print("=" * 40)

# Créer GeoDataFrame avec géométrie
geometry = [Point(lon, lat) for lat, lon in zip(df_actifs['latitude'], df_actifs['longitude'])]

gdf_actifs = gpd.GeoDataFrame(df_actifs, geometry=geometry, crs='EPSG:4326')

# Créer objet Exposures
exposures = Exposures()

# Remplir attributs obligatoires CLIMADA
exposures.gdf = gpd.GeoDataFrame({
    'value': df_actifs['valeur'].values,
    'latitude': df_actifs['latitude'].values,
    'longitude': df_actifs['longitude'].values,
    'region_id': pd.Categorical(df_actifs['commune']).codes + 1,  # Région = commune
    'impf_FL': 1,  # Impact function ID pour floods (sera définie plus tard)
    
    # Colonnes additionnelles pour analyse
    'secteur': df_actifs['secteur'].values,
    'type_actif': df_actifs['type_actif'].values,
    'commune': df_actifs['commune'].values,
    'vulnerabilite': df_actifs['vulnerabilite'].values,
    'densite_pop': df_actifs['densite_pop'].values,
    
    # Géométrie
    'geometry': geometry
})

# Métadonnées
exposures.tag.description = 'Exposition économique détaillée Abidjan - Formation DGE 2025'
exposures.tag.file_name = 'exposition_abidjan_detaillee.hdf5'
exposures.value_unit = 'FCFA'
exposures.ref_year = 2023
exposures.crs = 'EPSG:4326'

print(f"✅ GeoDataFrame créé: {len(exposures.gdf)} points")
print(f"🗺️ Système coordonnées: {exposures.crs}")
print(f"💰 Valeur totale: {exposures.gdf['value'].sum()/1e12:.2f} billions FCFA")
print(f"📅 Année référence: {exposures.ref_year}")

# Validation basique
if len(exposures.gdf) > 0:
    print(f"📊 Bounding box:")
    print(f"   Longitude: {exposures.gdf['longitude'].min():.3f} à {exposures.gdf['longitude'].max():.3f}")
    print(f"   Latitude: {exposures.gdf['latitude'].min():.3f} à {exposures.gdf['latitude'].max():.3f}")
    
    # Vérifier valeurs manquantes
    print(f"🔍 Valeurs manquantes:")
    for col in ['value', 'latitude', 'longitude']:
        n_missing = exposures.gdf[col].isna().sum()
        print(f"   {col}: {n_missing} manquantes")

In [ ]:
# Charger aléa créé précédemment pour assignation centroides
hazard_file = Path('data/hazards/precipitation_hazard_abidjan_formation.hdf5')

if hazard_file.exists():
    # Charger aléa existant
    hazard = Hazard.from_hdf5(hazard_file)
    print(f"📂 Aléa chargé: {hazard.size[0]} événements, {hazard.size[1]} centroides")
    
    # Assigner centroides exposition aux centroides aléa
    print("🎯 Assignation centroides exposition -> aléa...")
    exposures.assign_centroids(hazard)
    
    print(f"✅ Assignation terminée")
    print(f"📊 Centroides assignés: {exposures.gdf['centr_FL'].notna().sum()}/{len(exposures.gdf)}")
    
    # Statistiques d'assignation
    unassigned = exposures.gdf['centr_FL'].isna().sum()
    if unassigned > 0:
        print(f"⚠️ Points non assignés: {unassigned} ({unassigned/len(exposures.gdf)*100:.1f}%)")
    
else:
    print("⚠️ Fichier aléa non trouvé - création centroides simplifiés")
    
    # Créer centroides simplifiés pour test
    lon_min, lon_max = exposures.gdf['longitude'].min(), exposures.gdf['longitude'].max()
    lat_min, lat_max = exposures.gdf['latitude'].min(), exposures.gdf['latitude'].max()
    
    # Grille 20x20
    n_grid = 20
    lon_grid = np.linspace(lon_min, lon_max, n_grid)
    lat_grid = np.linspace(lat_min, lat_max, n_grid)
    lon_mesh, lat_mesh = np.meshgrid(lon_grid, lat_grid)
    
    # Créer hazard minimal pour assignation
    from climada.hazard import Centroids
    
    hazard = Hazard('FL')
    hazard.centroids.set_lat_lon(lat_mesh.flatten(), lon_mesh.flatten())
    
    # Événement minimal
    hazard.event_id = np.array([1])
    hazard.event_name = ['Test']
    hazard.date = pd.to_datetime(['2023-01-01']).values
    hazard.frequency = np.array([1.0])
    hazard.orig = np.array([True])
    hazard.intensity = np.ones((1, len(hazard.centroids.lat)))
    hazard.units = 'mm/day'
    
    exposures.assign_centroids(hazard)
    print(f"✅ Centroides simplifiés assignés")

# Validation finale CLIMADA
exposures.check()
print("✅ Objet Exposures validé CLIMADA!")

## 📊 Étape 5 : Visualisations et Analyses Spatiales

In [ ]:
# Visualisations complètes de l'exposition créée
fig = plt.figure(figsize=(20, 16))

# Configuration grille de subplots
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Carte générale exposition (valeur)
ax1 = fig.add_subplot(gs[0, 0])
scatter1 = ax1.scatter(exposures.gdf['longitude'], exposures.gdf['latitude'], 
                      c=exposures.gdf['value']/1e6, s=10, cmap='viridis', alpha=0.6)
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_title('Exposition Économique Totale')
ax1.grid(True, alpha=0.3)
cbar1 = plt.colorbar(scatter1, ax=ax1)
cbar1.set_label('Valeur (Millions FCFA)')

# 2. Exposition par secteur
ax2 = fig.add_subplot(gs[0, 1])
secteurs_uniques = df_actifs['secteur'].unique()
colors_secteur = plt.cm.Set3(np.linspace(0, 1, len(secteurs_uniques)))

for i, secteur in enumerate(secteurs_uniques):
    mask = exposures.gdf['secteur'] == secteur
    if mask.any():
        ax2.scatter(exposures.gdf.loc[mask, 'longitude'], 
                   exposures.gdf.loc[mask, 'latitude'],
                   c=[colors_secteur[i]], s=8, alpha=0.7, label=secteur.replace('_', ' ')[:12])

ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude')
ax2.set_title('Exposition par Secteur Économique')
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
ax2.grid(True, alpha=0.3)

# 3. Exposition par commune
ax3 = fig.add_subplot(gs[0, 2])
communes_uniques = df_actifs['commune'].unique()
colors_commune = plt.cm.tab10(np.linspace(0, 1, len(communes_uniques)))

for i, commune in enumerate(communes_uniques):
    mask = exposures.gdf['commune'] == commune
    if mask.any():
        ax3.scatter(exposures.gdf.loc[mask, 'longitude'], 
                   exposures.gdf.loc[mask, 'latitude'],
                   c=[colors_commune[i]], s=12, alpha=0.7, label=commune)

ax3.set_xlabel('Longitude')
ax3.set_ylabel('Latitude')
ax3.set_title('Exposition par Commune')
ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
ax3.grid(True, alpha=0.3)

# 4. Distribution valeurs (histogramme)
ax4 = fig.add_subplot(gs[1, 0])
ax4.hist(exposures.gdf['value']/1e6, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
ax4.set_xlabel('Valeur Actifs (Millions FCFA)')
ax4.set_ylabel('Fréquence')
ax4.set_title('Distribution des Valeurs d\'Exposition')
ax4.set_yscale('log')
ax4.grid(True, alpha=0.3)

# Statistiques sur le graphique
mean_val = exposures.gdf['value'].mean()/1e6
median_val = exposures.gdf['value'].median()/1e6
ax4.axvline(mean_val, color='red', linestyle='--', label=f'Moyenne: {mean_val:.1f}M')
ax4.axvline(median_val, color='orange', linestyle='--', label=f'Médiane: {median_val:.1f}M')
ax4.legend()

# 5. Exposition par secteur (barres)
ax5 = fig.add_subplot(gs[1, 1])
exposition_secteur = exposures.gdf.groupby('secteur')['value'].sum() / 1e12
bars = ax5.bar(range(len(exposition_secteur)), exposition_secteur.values, 
               color=colors_secteur[:len(exposition_secteur)])
ax5.set_xlabel('Secteur Économique')
ax5.set_ylabel('Exposition (Billions FCFA)')
ax5.set_title('Exposition Totale par Secteur')
ax5.set_xticks(range(len(exposition_secteur)))
ax5.set_xticklabels([s.replace('_', '\n') for s in exposition_secteur.index], 
                    rotation=45, ha='right')
ax5.grid(True, alpha=0.3)

# Ajouter valeurs sur barres
for bar, val in zip(bars, exposition_secteur.values):
    ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{val:.2f}T', ha='center', va='bottom', fontsize=8)

# 6. Exposition par commune (barres)
ax6 = fig.add_subplot(gs[1, 2])
exposition_commune = exposures.gdf.groupby('commune')['value'].sum() / 1e12
bars = ax6.bar(range(len(exposition_commune)), exposition_commune.values,
               color=colors_commune[:len(exposition_commune)])
ax6.set_xlabel('Commune')
ax6.set_ylabel('Exposition (Billions FCFA)')
ax6.set_title('Exposition Totale par Commune')
ax6.set_xticks(range(len(exposition_commune)))
ax6.set_xticklabels(exposition_commune.index, rotation=45, ha='right')
ax6.grid(True, alpha=0.3)

# 7. Densité d'exposition (carte de chaleur)
ax7 = fig.add_subplot(gs[2, 0])

# Créer grille pour heatmap
from scipy.stats import gaussian_kde

# Préparer données
points = np.vstack([exposures.gdf['longitude'], exposures.gdf['latitude']])
weights = exposures.gdf['value'].values

# KDE pondéré (approximation)
kde = gaussian_kde(points)
xi = np.linspace(exposures.gdf['longitude'].min(), exposures.gdf['longitude'].max(), 50)
yi = np.linspace(exposures.gdf['latitude'].min(), exposures.gdf['latitude'].max(), 50)
xi_mesh, yi_mesh = np.meshgrid(xi, yi)
positions = np.vstack([xi_mesh.ravel(), yi_mesh.ravel()])
density = kde(positions).reshape(xi_mesh.shape)

im = ax7.contourf(xi_mesh, yi_mesh, density, levels=15, cmap='Reds', alpha=0.8)
ax7.set_xlabel('Longitude')
ax7.set_ylabel('Latitude')
ax7.set_title('Densité d\'Exposition')
plt.colorbar(im, ax=ax7, label='Densité')

# 8. Relation densité population - exposition
ax8 = fig.add_subplot(gs[2, 1])
ax8.scatter(exposures.gdf['densite_pop'], exposures.gdf['value']/1e6, 
           alpha=0.5, s=10, c='purple')
ax8.set_xlabel('Densité Population (hab/km²)')
ax8.set_ylabel('Valeur Exposition (Millions FCFA)')
ax8.set_title('Relation Densité-Exposition')
ax8.set_xscale('log')
ax8.set_yscale('log')
ax8.grid(True, alpha=0.3)

# Ligne de tendance
from scipy.stats import linregress
log_density = np.log10(exposures.gdf['densite_pop'])
log_value = np.log10(exposures.gdf['value']/1e6)
slope, intercept, r_value, p_value, std_err = linregress(log_density, log_value)
x_trend = np.logspace(np.log10(exposures.gdf['densite_pop'].min()), 
                     np.log10(exposures.gdf['densite_pop'].max()), 100)
y_trend = 10**(slope * np.log10(x_trend) + intercept)
ax8.plot(x_trend, y_trend, 'r--', alpha=0.8, 
         label=f'R² = {r_value**2:.3f}')
ax8.legend()

# 9. Analyse vulnérabilité par zone
ax9 = fig.add_subplot(gs[2, 2])
vulnerabilite_ordre = ['faible', 'moyenne', 'elevee', 'tres_elevee']
vuln_colors = ['green', 'yellow', 'orange', 'red']

exposition_vuln = exposures.gdf.groupby('vulnerabilite')['value'].sum() / 1e12
# Réordonner selon ordre logique
exposition_vuln = exposition_vuln.reindex(vulnerabilite_ordre, fill_value=0)

bars = ax9.bar(range(len(exposition_vuln)), exposition_vuln.values, 
               color=vuln_colors)
ax9.set_xlabel('Niveau Vulnérabilité')
ax9.set_ylabel('Exposition (Billions FCFA)')
ax9.set_title('Exposition par Niveau de Vulnérabilité')
ax9.set_xticks(range(len(exposition_vuln)))
ax9.set_xticklabels([v.replace('_', ' ').title() for v in exposition_vuln.index])
ax9.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 💾 Étape 6 : Sauvegarde et Validation

In [ ]:
# Sauvegarde complète de l'exposition créée
output_dir = Path('data/exposures')
output_dir.mkdir(parents=True, exist_ok=True)

print("💾 SAUVEGARDE EXPOSITION ÉCONOMIQUE")
print("=" * 40)

# 1. Sauvegarder en format CLIMADA (HDF5)
exposure_file = output_dir / 'exposition_abidjan_detaillee.hdf5'
exposures.write_hdf5(exposure_file)
print(f"📂 Exposition CLIMADA: {exposure_file}")
print(f"   Taille: {exposure_file.stat().st_size / 1024 / 1024:.1f} MB")

# 2. Sauvegarder DataFrame complet (CSV)
csv_file = output_dir / 'exposition_abidjan_complete.csv'
df_actifs.to_csv(csv_file, index=False)
print(f"📊 Données complètes CSV: {csv_file}")

# 3. Sauvegarder GeoDataFrame (GeoJSON pour SIG)
geojson_file = output_dir / 'exposition_abidjan_spatial.geojson'
gdf_actifs.to_file(geojson_file, driver='GeoJSON')
print(f"🗺️ Données spatiales GeoJSON: {geojson_file}")

# 4. Statistiques de synthèse
stats_summary = {
    'creation_date': pd.Timestamp.now().isoformat(),
    'total_actifs': len(exposures.gdf),
    'valeur_totale_fcfa': float(exposures.gdf['value'].sum()),
    'valeur_moyenne_fcfa': float(exposures.gdf['value'].mean()),
    'valeur_mediane_fcfa': float(exposures.gdf['value'].median()),
    'bounding_box': {
        'lon_min': float(exposures.gdf['longitude'].min()),
        'lon_max': float(exposures.gdf['longitude'].max()),
        'lat_min': float(exposures.gdf['latitude'].min()),
        'lat_max': float(exposures.gdf['latitude'].max())
    },
    'secteurs': {
        secteur: {
            'nombre_actifs': int((exposures.gdf['secteur'] == secteur).sum()),
            'valeur_totale': float(exposures.gdf[exposures.gdf['secteur'] == secteur]['value'].sum())
        } for secteur in exposures.gdf['secteur'].unique()
    },
    'communes': {
        commune: {
            'nombre_actifs': int((exposures.gdf['commune'] == commune).sum()),
            'valeur_totale': float(exposures.gdf[exposures.gdf['commune'] == commune]['value'].sum())
        } for commune in exposures.gdf['commune'].unique()
    }
}

# Sauvegarder statistiques en JSON
import json
stats_file = output_dir / 'exposition_statistiques.json'
with open(stats_file, 'w', encoding='utf-8') as f:
    json.dump(stats_summary, f, indent=2, ensure_ascii=False)
print(f"📈 Statistiques JSON: {stats_file}")

# 5. Documentation technique
documentation = f"""
EXPOSITION ÉCONOMIQUE DÉTAILLÉE - ABIDJAN
Formation DGE CLIMADA - {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}

MÉTHODOLOGIE:
- Approche bottom-up basée données macro-économiques CI
- Génération stochastique contrainte par statistiques INS/BCEAO
- Géolocalisation par commune avec secteurs économiques
- Calibration sur PIB Abidjan 2023: {pib_abidjan/1e12:.2f} billions FCFA

CARACTÉRISTIQUES TECHNIQUES:
- Nombre total d'actifs: {len(exposures.gdf):,}
- Valeur totale: {exposures.gdf['value'].sum()/1e12:.2f} billions FCFA
- Zone couverte: District d'Abidjan (10 communes)
- Secteurs: {len(secteurs_economiques)} secteurs économiques
- Référence temporelle: 2023
- Système coordonnées: EPSG:4326 (WGS84)

RÉPARTITION SECTORIELLE:
"""

for secteur in secteurs_economiques.keys():
    n_actifs = (exposures.gdf['secteur'] == secteur).sum()
    valeur = exposures.gdf[exposures.gdf['secteur'] == secteur]['value'].sum()
    documentation += f"- {secteur}: {n_actifs} actifs, {valeur/1e12:.3f}T FCFA\n"

documentation += f"""
RÉPARTITION COMMUNALE:
"""

for commune in communes_abidjan.keys():
    n_actifs = (exposures.gdf['commune'] == commune).sum()
    valeur = exposures.gdf[exposures.gdf['commune'] == commune]['value'].sum()
    documentation += f"- {commune}: {n_actifs} actifs, {valeur/1e12:.3f}T FCFA\n"

documentation += f"""
VALIDATION MACROÉCONOMIQUE:
- PIB Abidjan officiel: {pib_abidjan/1e12:.2f}T FCFA
- Exposition générée: {exposures.gdf['value'].sum()/1e12:.2f}T FCFA
- Ratio exposition/PIB: {exposures.gdf['value'].sum()/pib_abidjan:.2f}
- Cohérence: {'✅ Acceptable' if 0.3 <= exposures.gdf['value'].sum()/pib_abidjan <= 1.5 else '⚠️ À vérifier'}

FICHIERS GÉNÉRÉS:
- {exposure_file.name}: Format CLIMADA (analyse risques)
- {csv_file.name}: Données tabulaires complètes
- {geojson_file.name}: Format SIG (cartographie)
- {stats_file.name}: Statistiques de synthèse

USAGE RECOMMANDÉ:
1. Charger avec: exposures = Exposures.from_hdf5('{exposure_file.name}')
2. Analyse spatiale: utiliser fichier GeoJSON
3. Analyse statistique: utiliser fichier CSV
4. Intégration modèles: utiliser objet CLIMADA

CONTACT SUPPORT:
formation.climada.dge@gouv.ci
"""

doc_file = output_dir / 'DOCUMENTATION_EXPOSITION_ABIDJAN.txt'
with open(doc_file, 'w', encoding='utf-8') as f:
    f.write(documentation)
print(f"📄 Documentation: {doc_file}")

print(f"\n✅ Sauvegarde complète terminée!")
print(f"📁 Dossier: {output_dir.absolute()}")
print(f"🎯 Prêt pour calculs d'impact CLIMADA")

## 🎯 Exercices Pratiques

### 🎯 Exercice 1: Modification de la distribution sectorielle
Modifiez les proportions sectorielles pour simuler différents scénarios économiques.

In [ ]:
# EXERCICE 1: Tester différents scénarios économiques
# Exemple: Augmenter part des services (+20%) et réduire industrie (-20%)

# Votre code ici:
# ---------------



# ---------------

### 🎯 Exercice 2: Analyse de sensibilité spatiale
Étudiez l'impact de différents rayons de dispersion spatiale sur la distribution finale.

In [ ]:
# EXERCICE 2: Tester différents paramètres de dispersion spatiale
# Modifiez le facteur 0.6 dans la génération des coordonnées

# Votre code ici:
# ---------------



# ---------------

### 🎯 Exercice 3: Intégration données de croissance
Ajoutez une projection 2030 basée sur les taux de croissance sectoriels.

In [ ]:
# EXERCICE 3: Créer exposition projetée 2030
# Utiliser les taux de croissance définis dans secteurs_economiques

# Votre code ici:
# ---------------



# ---------------

## 🎯 Résumé et Points Clés

### ✅ Compétences acquises:

1. **Modélisation exposition bottom-up**:
   - Intégration données macro-économiques officielles
   - Désagrégation spatiale et sectorielle
   - Calibration avec statistiques nationales
   - Validation cohérence macro/micro

2. **Géolocalisation économique**:
   - Distribution spatiale réaliste par commune
   - Prise en compte caractéristiques socio-économiques
   - Contraintes géographiques et densité
   - Différenciation urbain/périurbain

3. **Structure de données CLIMADA**:
   - Objet Exposures complet et validé
   - Assignation centroides pour calculs risques
   - Métadonnées et documentation technique
   - Formats d'export multiples (HDF5, CSV, GeoJSON)

4. **Analyse et validation**:
   - Statistiques descriptives multi-niveaux
   - Visualisations spatiales et sectorielles
   - Cohérence avec données de référence
   - Documentation pour reproductibilité

### 🔄 Prochaines étapes:
- TP4: Fonctions de dommage sectorielles calibrées
- TP5: Calcul impacts économiques complets
- TP6: Analyse coût-bénéfice mesures adaptation

### 📚 Ressources additionnelles:
- INS Côte d'Ivoire: www.ins.ci
- BCEAO: www.bceao.int
- Documentation CLIMADA: climada-python.readthedocs.io

### 📞 Support:
**formation.climada.dge@gouv.ci**

---
**📅 Formation DGE - CLIMADA Côte d'Ivoire 2025**